# Demo to demonstrate basic functionality for interacting with minio server
### Minio is wrapper allowing for hosting of Amazon S3-like storage on your own local server

# Docker host of minio
### Docker is a tool to make it super easy to create, deploy, and run applications, like a minio server, by creating and hosting the app on a virtual computer ('container').

In [9]:
# after installing docker, use docker command-line tool to start up container and save data in container to a local directory so that we can look at it on our host machine. This also allows for persistent data storage for minio server
# This is run in shell, not python
# docker run -p 9000:9000 --name minio1 -v C:\Users\rldun\data:/data minio/minio server /data
# This downloads and runs a docker image for minio, which is then mapped to localhost port 9000 for easy access on host. 

# Configure minio python environment

In [6]:
# again this is shell, not python
# conda create -n minio python=3.7

# Set environment as active and install some packages
# (source/conda) activate minio
# pip install tifffile matplotlib minio jupyter numpy

# Connect to docker minio server

In [15]:
from minio import Minio

# localhost is http hence secure=False... setting up https is possible but for another time
client = Minio('127.0.0.1:9000', access_key='minioadmin', secret_key='minioadmin', secure=False) 

# grab buckets
buckets = client.list_buckets()

# print buckets
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

boops 2020-02-14 02:51:27.026000+00:00
data 2020-03-02 21:16:20.278000+00:00


# Upload and download a single frame from a recording
### We can can work with files or byte streams. Working with byte streams means we upload/download the files in memory, whereas working with files goes straight to disk.
### To compute on minio file objects, we would download the file to disk, load it into memory, then do our computations
### To compute on byte streams, we load the byte stream directly into memory and do our computations

In [16]:
from minio.error import ResponseError
import io
import numpy as np
import tifffile as tf


# params
fname = 'C:/Users/rldun/Desktop/20200204-10-46-53_0-500.tiff'
bucketname = 'data'
objname = 'f0'

# load tiff
d = tf.imread(fname)

# connect to minio
client = Minio('127.0.0.1:9000', access_key='minioadmin', secret_key='minioadmin', secure=False)

# convert object to byte buffer
f = d[0,:,:]
b = f.tobytes()
bio = io.BytesIO(b)

# upload object
try:
    client.put_object(bucketname, objname, bio, len(b))
except ResponseError as err:
    print('Error during upload: {}'.format(err))

# download object
try:
    downloaded_data = client.get_object(bucketname, objname)
except ResponseError as err:
    print('Error during download: {}'.format(err))

# compare
res = downloaded_data.read()
new_f = np.frombuffer(res, dtype=f.dtype).reshape(f.shape)
if np.array_equal(f, new_f):
    print('Arrays are equal!')

Arrays are equal!


# Upload an entire recording as bytes
### Because we're working with byte streams, we can just convert our whole numpy array into a sequence of bytes for convenient i/o

In [17]:
import time

# params
fname = 'C:/Users/rldun/Desktop/20200204-10-46-53_0-500.tiff'
bucketname = 'data'
fname_root = '20200204-10-46-53_short/'

# load tiff
d = tf.imread(fname)

# connect to minio
client = Minio('127.0.0.1:9000', access_key='minioadmin', secret_key='minioadmin', secure=False)

tstart = time.time()
try:
    b = d.tobytes()
    bio = io.BytesIO(b)
    client.put_object(bucketname, fname_root + 'data', bio, len(b))
except ResponseError as err:
    print('Error during whole-file upload: {}'.format(err))

tend = time.time()

print('That took {} seconds to transfer recording of shape {}!'.format(tend - tstart, d.shape))



That took 5.914475440979004 seconds to transfer (500, 176, 602) frames!


# Grab individual frames out of entire byte-ified recording
### Using the appropriate offsets, we can get partial chunks out of a byte-object to extract individual frames
### Here we're loading a single frame from the data we uploaded above

In [18]:
# params
fname = 'C:/Users/rldun/Desktop/20200204-10-46-53_0-500.tiff'
bucketname = 'data'
fname_root = '20200204-10-46-53_short/'
objname = fname_root + 'data'

# load tiff
d = tf.imread(fname)

# connect to minio
client = Minio('127.0.0.1:9000', access_key='minioadmin', secret_key='minioadmin', secure=False)

# get number of bytes in single frame...
f_ndx_to_grab = 20
f = d[f_ndx_to_grab,:,:]
b = f.tobytes()
num_bytes_per_frame = len(b)

try:
    downloaded_data = client.get_partial_object(bucketname, objname, offset=f_ndx_to_grab*num_bytes_per_frame, length=num_bytes_per_frame)
except ResponseError as err:
    print('Error during download: {}'.format(err))

# compare downloaded chunk!
res = downloaded_data.read()
new_f = np.frombuffer(res, dtype=f.dtype).reshape(f.shape)
if np.array_equal(f, new_f):
    print('Arrays are equal!')


Arrays are equal!


# Unknowns
### What are the standout unknowns that come to mind?

In [ ]:
1) How to get docker on our buffalo NAS?
2) How to connect to hosted minio from external machine?
3) Best way to upload to NAS minio server? Local app?